In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
vitals = pd.read_csv('')

In [ ]:
vitals.info()

In [ ]:
vitals.sample(10)

In [ ]:
vitals['Risk Category'].value_counts()

In [ ]:
X = vitals.drop(['Patient ID','Timestamp','Risk Category'], axis=1)
y = vitals['Risk Category']

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
X['Gender'] = label_encoder.fit_transform(X['Gender'])
y = label_encoder.fit_transform(y)

In [ ]:
print(np.unique_counts(y))

In [ ]:
X_dl = X.values.reshape((X.shape[0], X.shape[1], 1))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_dl, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout
from sklearn.model_selection import train_test_split

In [ ]:
model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(X.shape[1], 1)),
    MaxPooling1D(2),
    Dropout(0.2),
    Conv1D(64, 3, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2)

In [ ]:
# Save model in HDF5 format
model.save('')
print("Model saved successfully!")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Extract accuracy values from training history
train_acc = history.history['accuracy']
val_acc = history.history.get('val_accuracy')  # may not exist

epochs = np.arange(1, len(train_acc) + 1)
mean_train = np.mean(train_acc)
mean_val = np.mean(val_acc) if val_acc else None

fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlim(0.5, len(epochs) + 0.5)
ax.set_ylim(0, 1.05)
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax.set_title('Human Vitals 1D-CNN Accuracy')
ax.grid(True)

line_train, = ax.plot([], [], label='Train Accuracy', color='blue')
line_val, = ax.plot([], [], label='Validation Accuracy', color='green') if val_acc else (None,)
mean_line_train = ax.axhline(mean_train, color='blue', linestyle='--', alpha=0.5, label='Train Mean')
mean_line_val = ax.axhline(mean_val, color='green', linestyle='--', alpha=0.5, label='Val Mean') if val_acc else None

ax.legend()

def init():
    line_train.set_data([], [])
    if val_acc:
        line_val.set_data([], [])
    return (line_train, line_val) if val_acc else (line_train,)

def update(frame):
    line_train.set_data(epochs[:frame + 1], train_acc[:frame + 1])
    if val_acc:
        line_val.set_data(epochs[:frame + 1], val_acc[:frame + 1])
        return line_train, line_val
    return line_train,

ani = FuncAnimation(fig, update, frames=len(epochs), init_func=init, blit=True, interval=100)

# Save as GIF or show
ani.save('', writer='pillow')

plt.close()  # to prevent double plot
HTML(ani.to_jshtml())

In [ ]:
sample = vitals.sample(1)

X_sample = sample.drop(['Patient ID','Timestamp','Risk Category'], axis=1)
y_sample = sample['Risk Category']

In [ ]:
print(y_sample)

In [ ]:
X_sample['Gender'] = label_encoder.fit_transform(X_sample['Gender'])
y_sample = label_encoder.fit_transform(y_sample)

X_sample_dl = X_sample.values.reshape((X_sample.shape[0], X_sample.shape[1], 1))

In [ ]:
pred_prob = model.predict(X_sample_dl)
pred_class = (pred_prob > 0.5).astype(int)

print(f"Prediction Probability: {pred_prob[0][0]:.4f}")
print("Risk Category Prediction:", "High Risk ⚠️" if pred_class[0][0] == 0 else "Low Risk ✅")